In [1]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File ‘00-start-eva-server.ipynb’ already there; not retrieving.

nohup eva_server > eva.log 2>&1 &
('01-11-2023 00:02:48 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF FastRCNNObjectDetector already exists, nothing added.\n', '01-11-2023 00:02:48 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF MVITActionRecognition already exists, nothing added.\n', '01-11-2023 00:02:48 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF Array_Count already exists, nothing added.\n')
('01-11-2023 00:02:48 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF Crop already exists, nothing added.\n', '01-11-2023 00:02:48 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF DummyObjectDetector already exists, nothing added.\n', '01-11-2023 00:02:48 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF DummyMultiObjectDetector already exists, nothing added.\n')
('01-11-2023 00:02:48 CRITICAL[server:server.py:start_server

In [3]:
cursor.execute("DROP TABLE test;")
response = cursor.fetch_all()
print(response)
cursor.execute("LOAD FILE 'test.png' INTO test;")
response = cursor.fetch_all()
print(response)
cursor.execute("""SELECT id FROM test WHERE id < 5""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                   0
0  Table Successfully dropped: test
@query_time: 0.03131404198938981
@status: ResponseStatus.SUCCESS
@batch: 
                                                 0
0  Video successfully added at location: test.png
@query_time: 0.024133584010996856
@status: ResponseStatus.SUCCESS
@batch: 
    test.id
0        0
@query_time: 0.01936524998745881


In [4]:
cursor.execute("""DROP UDF CustomModel;""")
response = cursor.fetch_all()
print(response)

cursor.execute("""CREATE UDF IF NOT EXISTS 
                  CustomModel
                  INPUT (frame NDARRAY UINT8(3, ANYDIM, ANYDIM)) 
                  OUTPUT (results NDARRAY FLOAT32(ANYDIM, ANYDIM)) 
                  TYPE  Classification 
                  IMPL 'CustomModel.py';
        """)
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                       0
0  UDF CustomModel successfully dropped
@query_time: 0.02271308400668204
@status: ResponseStatus.SUCCESS
@batch: 
                                                      0
0  UDF CustomModel successfully added to the database.
@query_time: 0.29801754200889263


In [5]:
cursor.execute("""SELECT id, CustomModel(data)
                  FROM test WHERE id < 10""")


response = cursor.fetch_all()

print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: expected 5D input (got 4D input)
